In [ ]:
import math
import os
import shutil
from collections import Counter

def reorg_dog_data(data_dir, label_file, train_dir, test_dir, input_dir,
                   valid_ratio):
    # 读取训练数据标签。
    with open(os.path.join(data_dir, label_file), 'r') as f:
        # 跳过文件头行（栏名称）。
        lines = f.readlines()[1:]
        tokens = [l.rstrip().split(',') for l in lines]
        idx_label = dict(((idx, label) for idx, label in tokens))
    labels = set(idx_label.values())

    num_train = len(os.listdir(os.path.join(data_dir, train_dir)))
    # 训练集中数量最少一类的狗的数量。
    min_num_train_per_label = (
        Counter(idx_label.values()).most_common()[:-2:-1][0][1])
    # 验证集中每类狗的数量。
    num_valid_per_label = math.floor(min_num_train_per_label * valid_ratio)
    label_count = dict()

    def mkdir_if_not_exist(path):
        if not os.path.exists(os.path.join(*path)):
            os.makedirs(os.path.join(*path))

    # 整理训练和验证集。
    for train_file in os.listdir(os.path.join(data_dir, train_dir)):
        idx = train_file.split('.')[0]
        label = idx_label[idx]
        mkdir_if_not_exist([data_dir, input_dir, 'train_valid', label])
        shutil.copy(os.path.join(data_dir, train_dir, train_file),
                    os.path.join(data_dir, input_dir, 'train_valid', label))
        if label not in label_count or label_count[label] < num_valid_per_label:
            mkdir_if_not_exist([data_dir, input_dir, 'valid', label])
            shutil.copy(os.path.join(data_dir, train_dir, train_file),
                        os.path.join(data_dir, input_dir, 'valid', label))
            label_count[label] = label_count.get(label, 0) + 1
        else:
            mkdir_if_not_exist([data_dir, input_dir, 'train', label])
            shutil.copy(os.path.join(data_dir, train_dir, train_file),
                        os.path.join(data_dir, input_dir, 'train', label))

    # 整理测试集。
    mkdir_if_not_exist([data_dir, input_dir, 'test', 'unknown'])
    for test_file in os.listdir(os.path.join(data_dir, test_dir)):
        shutil.copy(os.path.join(data_dir, test_dir, test_file),
                    os.path.join(data_dir, input_dir, 'test', 'unknown'))



In [1]:
data_dir = u"/media/yijie/文档/dataset/ImageNetDog"
label_file = 'labels.csv'
train_dir = 'train'
test_dir = 'test'
input_dir = 'train_valid_test'
batch_size = 4
valid_ratio = 0.1
#reorg_dog_data(data_dir, label_file, train_dir, test_dir, input_dir,
#                  valid_ratio)

In [2]:
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
from mxnet.gluon.data import vision
import numpy as np

def transform_train(data, label):
    im = image.imresize(data.astype('float32') / 255, 224, 224)
    auglist = image.CreateAugmenter(data_shape=(3, 96, 96), resize=0,
                        rand_crop=True, rand_resize=True, rand_mirror=True,
                        mean=None, std=None,
                        brightness=0, contrast=0,
                        saturation=0, hue=0,
                        pca_noise=0, rand_gray=0, inter_method=2)
    for aug in auglist:
        im = aug(im)
    # 将数据格式从"高*宽*通道"改为"通道*高*宽"。
    im = nd.transpose(im, (2,0,1))
    return (im, nd.array([label]).asscalar().astype('float32'))

def transform_test(data, label):
    im = image.imresize(data.astype('float32') / 255, 224, 224)
    im = nd.transpose(im, (2,0,1))
    return (im, nd.array([label]).asscalar().astype('float32'))

In [8]:
input_str = "/media/yijie/文档/dataset/ImageNetDog/train_valid_test/"

# 读取原始图像文件。flag=1说明输入图像有三个通道（彩色）。
train_ds = vision.ImageFolderDataset(input_str + 'train', flag=1,
                                     transform=transform_train)
valid_ds = vision.ImageFolderDataset(input_str + 'valid', flag=1,
                                     transform=transform_test)
train_valid_ds = vision.ImageFolderDataset(input_str + 'train_valid',
                                           flag=1, transform=transform_train)
test_ds = vision.ImageFolderDataset(input_str + 'test', flag=1,
                                     transform=transform_test)
loader = gluon.data.DataLoader
train_data = loader(train_ds, batch_size, shuffle=True, last_batch='keep')
valid_data = loader(valid_ds, batch_size, shuffle=True, last_batch='keep')
train_valid_data = loader(train_valid_ds, batch_size, shuffle=True,
                          last_batch='keep')
test_data = loader(test_ds, batch_size, shuffle=False, last_batch='keep')
# 交叉熵损失函数。
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

(
[[[ 0.95815331  0.96644461  0.95673639 ...,  0.44268918  0.44551545
    0.44031864]
  [ 0.84264708  0.8605125   0.86897981 ...,  0.42965305  0.42990196
    0.42946154]
  [ 0.90359604  0.91752452  0.90556836 ...,  0.41973042  0.41629517
    0.41495103]
  ..., 
  [ 0.63869488  0.65001154  0.60163146 ...,  0.67216229  0.65136719
    0.63767618]
  [ 0.61072308  0.64387256  0.63322616 ...,  0.73697925  0.66449147
    0.59218371]
  [ 0.66623014  0.61869645  0.63386953 ...,  0.72000998  0.65940565
    0.60916817]]

 [[ 0.95123315  0.95749843  0.94047564 ...,  0.45445389  0.45728019
    0.45208335]
  [ 0.82095593  0.83882123  0.84471887 ...,  0.44141778  0.44166666
    0.44122627]
  [ 0.86903727  0.88296568  0.87165672 ...,  0.43149513  0.42805988
    0.4267157 ]
  ..., 
  [ 0.60340071  0.61471736  0.56633735 ...,  0.63294661  0.61215156
    0.5984605 ]
  [ 0.5754289   0.60857844  0.59793198 ...,  0.6959253   0.62343746
    0.55112982]
  [ 0.63093597  0.58340228  0.59857541 ...,  0.66902959 

In [11]:
from mxnet import init
from mxnet.gluon.model_zoo import vision as models

pretrained_net = models.resnet152_v1(pretrained=True)
finetune_net = models.resnet152_v1(classes=120)
finetune_net.features = pretrained_net.features
finetune_net.output.initialize(init.Xavier())
finetune_net.output

AttributeError: 'ResNetV1' object has no attribute 'output'

In [12]:
import datetime
import utils
import mxnet as mx
from mxnet import init
from visual_visdom import Visualizer

vis = Visualizer(env='demo')


def train(net, ctx, train_data,batch_size=8, epochs=100, learning_rate=0.01, wd=0.001,print_batches=None):
    vis.vis.env = 'demo'
    net.collect_params().reset_ctx(ctx)
    net.hybridize()
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {
        'learning_rate': learning_rate, 'wd': wd})
    
    prev_time = datetime.datetime.now()
    for epoch in range(epochs):
        train_loss = 0.0
        if epoch > 0 and epoch % 10 == 0:
            trainer.set_learning_rate(trainer.learning_rate * 0.4)
        for data, label in train_data:
            label = label.as_in_context(ctx)
            with autograd.record():
                output = net(data.as_in_context(ctx))
                loss = softmax_cross_entropy(output, label)
            loss.backward()
            trainer.step(batch_size)
            train_loss += nd.mean(loss).asscalar()
        cur_time = datetime.datetime.now()
        
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        Train_loss_vis = train_loss / len(train_data)
        epoch_str = ("Epoch %d. Train loss: %f, "
                         % (epoch, Train_loss_vis))
        vis.plot('train_loss',Train_loss_vis)
        prev_time = cur_time
        print(epoch_str + time_str + ', lr ' + str(trainer.learning_rate))
        



In [13]:
import numpy as np
ctx = utils.try_gpu()
train(finetune_net, ctx,train_valid_data)

outputs = []
for data, label in test_data:
    output = nd.softmax(finetune_net(data.as_in_context(ctx)))
    outputs.extend(output.asnumpy())
ids = sorted(os.listdir(os.path.join(data_dir, input_dir, 'test/unknown')))
with open('submission.csv', 'w') as f:
    f.write('id,' + ','.join(train_valid_ds.synsets) + '\n')
    for i, output in zip(ids, outputs):
        f.write(i.split('.')[0] + ',' + ','.join(
            [str(num) for num in output]) + '\n')

ValueError: Cannot reset context for Parameter resnetv15_dense0_weight because it has not been initialized.